In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
import requests
BASE_URL = 'https://shumi.info'
REQUEST_URL_CATEGORY = 'https://shumi.info/category'
res_category = requests.get(REQUEST_URL_CATEGORY)

soup_category = BeautifulSoup(res_category.text, 'html.parser')

In [64]:
#スクレイピングの参照範囲をaタグに狭めている
hobby_category = soup_category.select('ul.list.cf li a')

In [65]:
#全ページ分の趣味名を蓄積するためのリストを空で用意する
hobby_data = []

In [66]:
import re
import time

#スクレイピングの実行
for a_tag in hobby_category: #hobby_categoryの範囲内でできることを次の段落で記述
    print(f"カテゴリ取得中: {a_tag.text.strip()}")

    url_category = a_tag.get("href") #aタグに狭めた参照範囲からhrefを取り出す
    res_category_name = requests.get(BASE_URL+url_category, timeout = 5) #各趣味カテゴリのURLをループ生成    
    time.sleep(1)
    soup_hobby_category_url = BeautifulSoup(res_category_name.text, 'html.parser') #parse処理
    hobby_names = soup_hobby_category_url.select('a p.ttl') #各趣味カテゴリに属する名前を抽出
    for tag in hobby_names: #hobby_namesの範囲内でできることを次の段落で記述
        url_hobby = tag.find_parent("a").get("href")
        full_url = BASE_URL + url_hobby
        print(f"  → 趣味ページアクセス中: {full_url}")  # ★どこにアクセスしているか表示

        try:
            res_hobby_name = requests.get(full_url, timeout=5)
            time.sleep(1)
            soup_hobby_name = BeautifulSoup(res_hobby_name.text, 'html.parser')  # parse処理

            p_tags = soup_hobby_name.find_all("p")
            hobby_description = " ".join(p.text.strip() for p in p_tags if p.text.strip())
            hobby_description = re.sub(r"^あなたの人生を変える趣味、きっと見つかる。\s*文字サイズ：\s*", "", hobby_description)
        
            hobby_data.append({
                'カテゴリ': a_tag.text.strip(),
                '趣味名': tag.text.strip(),
                "趣味詳細説明文": hobby_description
            })
            print(f"    ✅ 成功: {tag.text.strip()}")
        
        except Exception as e:
            print(f"    ⚠️ 失敗: {full_url} → {e}")
            continue

カテゴリ取得中: スポーツ
  → 趣味ページアクセス中: https://shumi.info/407
    ✅ 成功: アーチェリー
  → 趣味ページアクセス中: https://shumi.info/382
    ✅ 成功: 水泳
  → 趣味ページアクセス中: https://shumi.info/410
    ✅ 成功: テニス
  → 趣味ページアクセス中: https://shumi.info/406
    ✅ 成功: 乗馬
  → 趣味ページアクセス中: https://shumi.info/107
    ✅ 成功: ボルダリング
  → 趣味ページアクセス中: https://shumi.info/392
    ✅ 成功: 合気道
  → 趣味ページアクセス中: https://shumi.info/381
    ✅ 成功: バドミントン
  → 趣味ページアクセス中: https://shumi.info/385
    ✅ 成功: スキー
  → 趣味ページアクセス中: https://shumi.info/567
    ✅ 成功: 草野球
  → 趣味ページアクセス中: https://shumi.info/376
    ✅ 成功: フィギュアスケート
  → 趣味ページアクセス中: https://shumi.info/352
    ✅ 成功: スケートボード
  → 趣味ページアクセス中: https://shumi.info/348
    ✅ 成功: スノーボード
  → 趣味ページアクセス中: https://shumi.info/256
    ✅ 成功: 剣道
  → 趣味ページアクセス中: https://shumi.info/232
    ✅ 成功: サーフィン
  → 趣味ページアクセス中: https://shumi.info/50
    ✅ 成功: ゴルフ
  → 趣味ページアクセス中: https://shumi.info/149
    ✅ 成功: ロードバイク
  → 趣味ページアクセス中: https://shumi.info/58
    ✅ 成功: 卓球
  → 趣味ページアクセス中: https://shumi.info/48
    ✅ 成功: バレーボール
  → 趣味ページ

KeyboardInterrupt: 

In [40]:
#あとで消す
print(len(hobby_data))

459


In [46]:
#ファイルの出力
df = pd.DataFrame(hobby_data)
df.to_csv("hobby_data.csv", index=False, encoding = "utf-8-sig")

In [47]:
import pandas as pd
df = pd.DataFrame(hobby_data)

In [48]:
#趣味名の重複が多いので、削除する
# ステップ① 空白除去
df["趣味名"] = df["趣味名"].astype(str).str.strip()

# ステップ② 重複削除して独立コピー作成
df_unique = df.drop_duplicates(subset="趣味名", keep="first").copy()

# ステップ③ 「アーチェリー」だけを完全一致で抽出（空白・改行など除去して比較）
cleaned_hobby_names = df_unique["趣味名"].astype(str).str.replace(r"\s+", "", regex=True).str.strip()
archery_indexes = df_unique[cleaned_hobby_names == "アーチェリー"].index

# ステップ④ 最初の1件を残して他は削除
df_unique2 = df_unique.drop(index=archery_indexes[1:]).reset_index(drop=True)

In [49]:
#あとで消す
df_unique2.to_csv("hobby_data_cleaned.csv", index=False, encoding = "utf-8-sig")

In [50]:
#マッチング作業
#まずpowershellでこちらを実行する　pip install sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import os
os.environ["HF_HOME"] = "./hf_cache"  # 別フォルダにキャッシュを保存

In [51]:
#ファイルの出力
#df_unique2.to_csv("hobby_data_unique2.csv", index=False)

In [52]:
# データの読み込み
#df_unique2 = pd.read_csv("hobby_data_unique2.csv")  # ←適宜ファイル名に置き換えてください
#df_unique2["趣味詳細説明文"] = df_unique2["趣味詳細説明文"].fillna("")

In [53]:
# 日本語対応のマルチリンガルモデル（安定版）
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
synonym_dict = {
    "体幹を鍛える": [
        "体幹", "バランス", "姿勢強化",
        "インナーマッスル", "軸が安定する", "安定感", "体の中心", "体幹トレーニング", "コア", "胴体強化", "体のブレが減る", "ケガ予防", "姿勢改善"
    ],
    "姿勢がよくなる": [
        "美姿勢", "姿勢改善", "背筋が伸びる",
        "猫背解消", "背筋", "自然な立ち姿", "自信がつく", "姿勢美人", "印象が良くなる", "健康的", "体が整う", "見た目が若返る", "骨盤矯正"
    ],
    "代謝が上がる": [
        "ダイエット", "脂肪燃焼", "体温上昇",
        "痩せる", "汗をかく", "基礎代謝", "内臓脂肪燃焼", "エネルギー消費", "太りにくくなる", "デトックス", "筋トレ効果", "血流促進", "引き締め"
    ],
    "リラックス": [
        "ストレス解消", "安らぐ", "落ち着く",
        "癒し", "深呼吸", "緊張がほぐれる", "穏やか", "気持ちが和らぐ", "安心感", "癒される時間", "副交感神経", "気持ちが整う", "気分転換"
    ],
    "癒される": [
        "癒し", "心が和む", "ほっこりする",
        "優しい時間", "あたたかい気持ち", "穏やかになる", "心が休まる", "精神的な回復", "心地よい", "優しい空気", "穏やかな風景", "満たされる", "癒し効果"
    ],
    "ワクワクする": [
        "興奮", "テンションが上がる", "ドキドキ",
        "冒険心", "期待感", "楽しい予感", "ハイテンション", "わくわく感", "童心にかえる", "躍動感", "高揚感", "やってみたい気持ち", "好奇心が刺激される"
    ],
    "集中する": [
        "没頭", "ゾーンに入る", "集中力が増す",
        "時間を忘れる", "夢中になる", "一点集中", "余計なことを忘れる", "作業効率アップ", "雑念が消える", "集中状態", "集中タイム", "深く入り込む", "生産性向上"
    ],
    "非日常感を味わう": [
        "刺激的", "冒険", "日常から離れる",
        "特別感", "旅行気分", "新しい体験", "旅に出る", "リゾート感", "新鮮さ", "異世界", "気分転換", "脱日常", "未知との出会い"
    ],
    "表現力が身につく": [
        "アウトプット力", "感情表現", "自己表現",
        "発信力", "伝える力", "創作活動", "話し方が上手くなる", "伝達能力", "感性が磨かれる", "自分を表す", "創造的思考", "魅せ方を学ぶ", "パフォーマンス力"
    ],
    "観察力が鍛える": [
        "洞察", "気づき", "注意深くなる",
        "観察眼", "細部に気づく", "よく見る力", "鋭い視点", "注意力", "客観視", "冷静な分析", "周囲をよく見る", "観察力アップ", "データに強くなる"
    ],
    "創造力を育てる": [
        "クリエイティブ", "想像力", "アイデア",
        "発想力", "ものづくり", "企画力", "アート思考", "柔軟な発想", "創作力", "イメージ力", "新しい視点", "ゼロから作る", "発明"
    ],
    "達成感を味わう": [
        "やりがい", "目標達成", "満足感",
        "チャレンジ成功", "自分に勝つ", "完遂感", "達成した気持ち", "報われる", "頑張った成果", "やった！と思える", "自信がつく", "満たされる", "積み重ねの成果"
    ],
    "文化に触れる": [
        "伝統文化", "アート", "知的好奇心",
        "歴史を学ぶ", "海外の風習", "芸術鑑賞", "クラシック", "古典", "民俗芸能", "日本文化", "美術館", "伝統芸能", "文化的教養"
    ],
    "自然と関わる": [
        "自然体験", "屋外", "アウトドア",
        "緑に触れる", "森林浴", "キャンプ", "自然観察", "四季を感じる", "田舎体験", "野外活動", "花や植物", "地球とつながる", "ナチュラル志向"
    ],
    "一人で楽しむ": [
        "ソロ", "マイペース", "自分の時間",
        "一人旅", "内省的", "集中できる", "人に気を使わない", "自分軸", "孤独を楽しむ", "ひとり趣味", "自由時間", "自己充電", "気まま"
    ],
    "仲間と楽しむ": [
        "グループ", "交流", "コミュニティ",
        "チーム活動", "一緒に笑う", "つながり", "共同作業", "友達と楽しむ", "人との関係", "対話", "社交性", "仲間意識", "連帯感"
    ],
    "初期費用が安い": [
        "低コスト", "安価", "コスパがいい",
        "節約できる", "お金がかからない", "無料ではじめられる", "格安", "お手頃価格", "続けやすい", "経済的", "自己投資が少ない", "出費が少ない", "維持費が安い"
    ],
    "自宅でできる": [
        "家でできる", "在宅", "家時間",
        "家から出なくていい", "室内", "雨でもできる", "防音不要", "家の中", "オンライン", "ホームエクササイズ", "自宅趣味", "省スペース", "リモート対応"
    ],
    "気軽に始められる": [
        "初心者OK", "始めやすい", "敷居が低い",
        "すぐにできる", "準備が簡単", "誰でもできる", "ハードルが低い", "初期知識不要", "導入が楽", "とっつきやすい", "カジュアル", "気負わずできる", "始めやすさ"
    ]
}

In [ ]:
user_input = ["あなたは趣味を通じて代謝が上がる、集中する、達成感を味わうことを望んでいます。またそれは仲間と楽しむことができ、なおかつ気軽に始められるものであると、より満足感が高まります。"]

In [ ]:
# ユーザーの回答に含まれるキーワード
user_keywords = ["代謝が上がる", "集中する", "達成感を味わう", "仲間と楽しむ", "気軽に始められる"]

In [57]:
# マッチングのルールを作成（ユーザーが回答したキーワードをカバーしている数が多いほどポイントが高い、更にすべてカバーしている場合にマッチしたキーワードの数が多ければボーナスポイント追加）
def hybrid_keyword_score(user_input, user_keywords, synonym_dict):
    covered_count = 0
    total_match_count = 0

    for keyword in user_keywords:
        synonyms = [keyword] + synonym_dict.get(keyword, [])
        match_count = 0
        for syn in synonyms:
            if syn in user_input:
                match_count +=1
        if match_count > 0:
            covered_count += 1
            total_match_count += match_count
        else:
            total_match_count += 0

    base_score = covered_count

    if covered_count == len(user_keywords):
        bonus_score = (total_match_count - covered_count) * 0.1
    else:
        bonus_score = 0
    
    return base_score + bonus_score

In [58]:
#上記で作成したマッチングルールを適用して"hybrid_score"としてカラム追加する
df_unique2["hybrid_score"] = df_unique2["趣味詳細説明文"].astype(str).apply(
    lambda x: hybrid_keyword_score(user_input, user_keywords, synonym_dict)
)

In [59]:
#"hybrid_score"を降順に並べたうえで、TOP3を抽出する
def get_top_n_hobby_name(df,column_name, n=3):
    return df.sort_values(by=column_name, ascending=False).head(n)

top_hobbies = get_top_n_hobby_name(df_unique2, "hybrid_score", n=3)

print(top_hobbies[["趣味名","hybrid_score"]])

         趣味名  hybrid_score
0      ライブ観戦             0
226    電子ピアノ             0
234  ジャズサックス             0


In [24]:
print(len(df_unique2))

1


In [61]:
import sqlite3
import pandas as pd

# 毎回スクレイピングせず、事前保存されたDBから読み込むだけ！
conn = sqlite3.connect("hobby_data_cleaned.db")
df = pd.read_sql_query("SELECT * FROM hobbies", conn)
conn.close()

# 必要ならこの df に対してマッチング処理などを行う
print(df.head())  # データ確認

   カテゴリ     趣味名                                            趣味詳細説明文  \
0  スポーツ   ライブ観戦  ライブは"鑑賞する"、"聴く"というよりも、音楽を楽しみつつ、その場の臨場感も楽しむことから...   
1  スポーツ  アーチェリー  みなさん、アーチェリーといえば、何を思い浮かべますか？\n2004年アテネオリンピックで、「...   
2  スポーツ      水泳  水泳と言えば、子供に習わせたい習い事の上位にランクインされる人気のスポーツのひとつ。\nしか...   
3  スポーツ     テニス  テニスというと最近では錦織選手を筆頭に、世界で活躍する選手が増えてきました。\nもとより人気...   
4  スポーツ      乗馬  乗馬と聞くと、『お金持ちや貴族など、特別な人のためのスポーツ』そんな印象を持っていませんか？...   

   hybrid_score  
0             0  
1             0  
2             0  
3             0  
4             0  
